In [19]:
import joblib
import os
from API.utils import *
import pandas as pd
import numpy as np
os.listdir('./model')

['ANN(58_to_sp162).pkl', 'SVR(5_to_23).pkl', 'transformer(23_to_54).pkl']

# load pretrained model

In [20]:
A = joblib.load('./model/SVR(5_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')

# input data

In [21]:
df = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).head(1)
y_test = df.iloc[:,4:]
df['P'] = 100 - df['N+A']
X_test = df[['T10','T50','T90','N+A','P']]
X_test

,T10,T50,T90,N+A,P
0,98.6,119.2,146.1,40.133,59.867


# step1

In [22]:
y_pred23 = A.predict(X_test)
y_pred23 = pd.DataFrame(y_pred23,columns=y_test.columns)
print(y_pred23.sum(axis=1))
y_pred23.head()

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,...,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.203686,0.819795,0.162816,3.60144,2.287253,3.558247,0.671907,8.028073,6.381046,9.598324,...,7.060207,5.897693,5.231254,9.390761,4.992742,3.516019,1.14882,5.289111,0.680566,0.697045


# step 2

In [23]:
y_pred54 = B(y_pred23)
print(y_pred54.sum(axis=1))
y_pred54.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,...,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.04099,0.08198,0.68043,1.203686,0.016396,0.162816,2.241508,...,1.319701,3.079975,0.680566,0.963369,0.651806,0.182203,0.037895,0.003248,0.007344,0.0


# step3

In [24]:
case_col = ['Case Conditions_Feed Rate to C013 (Input)', 
            'Case Conditions_C7+ in Light End (Input)', 
            'Case Conditions_Heart Cut Prod. Rate (Input)', 
            'Case Conditions_C6P- in Heart Cut (Input)']
case = pd.DataFrame(index=[0],columns=case_col)
case.iloc[0,:] = [10,2.5,100,1]

y_pred58 = pd.concat([case,y_pred54],axis=1)
y_pred58

,Case Conditions_Feed Rate to C013 (Input),Case Conditions_C7+ in Light End (Input),Case Conditions_Heart Cut Prod. Rate (Input),Case Conditions_C6P- in Heart Cut (Input),O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,...,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,10,2.5,100,1,0.0,0.0,0.0,0.04099,0.08198,0.68043,...,1.319701,3.079975,0.680566,0.963369,0.651806,0.182203,0.037895,0.003248,0.007344,0.0


In [25]:
sp162 = C.predict(y_pred58)
print(sp162.sum(axis=1))
sp162

0    54.0
dtype: float32


,Individual Component to Light End Split Factor_Oxygen,Individual Component to Heart Cut Split Factor_Oxygen,Individual Component to Heavy End Split Factor_Oxygen,Individual Component to Light End Split Factor_Methane,Individual Component to Heart Cut Split Factor_Methane,Individual Component to Heavy End Split Factor_Methane,Individual Component to Light End Split Factor_Ethane,Individual Component to Heart Cut Split Factor_Ethane,Individual Component to Heavy End Split Factor_Ethane,Individual Component to Light End Split Factor_Propane,...,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Light End Split Factor_n-Dodecane,Individual Component to Heart Cut Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Light End Split Factor_n-Hexylbenzene,Individual Component to Heart Cut Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Light End Split Factor_n-Tridecane,Individual Component to Heart Cut Split Factor_n-Tridecane,Individual Component to Heavy End Split Factor_n-Tridecane
0,0.999803,0.0001,0.000097,0.999925,0.000052,0.000023,0.999828,0.000041,0.000131,0.997634,...,0.455053,0.003947,0.52617,0.469883,0.001384,0.522552,0.476064,0.000071,0.000243,0.999686
